In [66]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd, mysql.connector
import xgboost, textblob, string
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.preprocessing import MultiLabelBinarizer

import keras
from keras.layers import Embedding, Flatten, Dense, GlobalMaxPool1D, Dropout
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [3]:
df1 = pd.read_excel('\\Users\\Dan\\Documents\\Python\\Data\\Flix_products\\SamsungReport.xlsx','Features', keep_default_na=False)

In [4]:
df1[['title', 'html']] = df1[['title', 'html']].astype(str)
df1.head()

,product_id,title,html
0,628714,xbox-supertemplate,xbox-supertemplate1111 kieskrreurig
1,653143,contrast enhancer,Applies one million criteria for contrast yet ...
2,653143,colour optimiser,Preserves the original level of brightness whi...
3,653143,detail enhancer,Selectively sharpens edges so that you see thi...
4,653143,3way burn-in protection,3 Types of Anti-Burn Program\nScreen burn-in i...


In [5]:
df1.apply(lambda x: x.count(), axis=0)

product_id    49215
title         49215
html          49215
dtype: int64

In [6]:
# concatenate title and html for the same product_id into one cell
df2 = df1.groupby('product_id')['title','html'].agg(lambda x: ' '.join(x)).reset_index()
df2['texts'] = df2['title'] + " " + df2['html']
df3 = df2.groupby('product_id')['texts'].agg(lambda x: ' '.join(x)).reset_index()

In [7]:
df3.shape

(6036, 2)

In [8]:
df3.head(2)

,product_id,texts
0,628714,xbox-supertemplate xbox-supertemplate1111 kies...
1,653143,contrast enhancer colour optimiser detail enha...


In [10]:
# merge some product details
products = pd.read_csv('\\Users\\Dan\\Documents\\Python\\Data\\products.csv', sep='\t')
df3 = pd.merge(df3, products[['product_id', 'flix_parent_category', 'flix_subCategory1']],
                 on='product_id', how='left')

C:\Users\Dan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
df3.astype(str).groupby('flix_parent_category')['product_id'].count()

flix_parent_category
Cameras                      276
Computing/Gaming            1442
Health & Beauty                2
Home Appliances              703
Home Entertainment          1076
Office Needs                   4
Other                          7
Phone/Mobiles               1489
Small Gadgets/Appliances      21
nan                         1016
Name: product_id, dtype: int64

In [12]:
# drop products with unassigned categories
df3.dropna(subset=['flix_parent_category'], inplace=True)

In [14]:
# separate description and product categories
texts = df3['texts']
texts.shape

(5020,)

In [39]:
maxlen = 100   # cuts off after 100 words
training_samples = 3000    # trains on 200 samples
#validation_samples = 10000   #  validates on 10,000 samples
max_words = 1000    # considers only the top 10,000 words in the dataset

In [40]:
# converts each description to a list of integers considering only the max_words
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)  

In [43]:
data = pad_sequences(sequences, maxlen=maxlen)   # each  tokenized review is cut to maxlen (100 words or padded 0 if less)

In [41]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 15497 unique tokens.


In [68]:
# encode the target labels
encoder = preprocessing.LabelEncoder()
encoded_labels = encoder.fit_transform(df3['flix_parent_category'])
set(labels)

{0, 1, 2, 3, 4, 5, 6, 7, 8}

In [69]:
labels = keras.utils.to_categorical(encoded_labels)
labels.shape

(5161, 9)

In [108]:
# mlb = MultiLabelBinarizer()
# labels = mlb.fit_transform(encoded_labels)
# labels.shape

In [72]:
labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (5161, 100)
Shape of label tensor: (5161, 9)


In [73]:
indices = np.arange(data.shape[0])  
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

In [74]:
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: ]
y_val = labels[training_samples: ]

In [75]:
#set(y_val), type(y_train)

TypeError: unhashable type: 'numpy.ndarray'

In [126]:
# Training the same model without pretrained word embeddings
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

embedding_dim = 100
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(9, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 100, 100)          100000    
_________________________________________________________________
flatten_11 (Flatten)         (None, 10000)             0         
_________________________________________________________________
dense_24 (Dense)             (None, 64)                640064    
_________________________________________________________________
dense_25 (Dense)             (None, 9)                 585       
Total params: 740,649
Trainable params: 740,649
Non-trainable params: 0
_________________________________________________________________


In [76]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model.save_weights('from scratch_model1')

Instructions for updating:
Use tf.cast instead.
Train on 4000 samples, validate on 1161 samples
Epoch 1/10
4000/4000 [==============================] - 2s 591us/step - loss: 0.8310 - acc: 0.7668 - val_loss: 0.3951 - val_acc: 0.8880
Epoch 2/10
4000/4000 [==============================] - 2s 506us/step - loss: 0.2059 - acc: 0.9465 - val_loss: 0.2858 - val_acc: 0.9216
Epoch 3/10
4000/4000 [==============================] - 2s 502us/step - loss: 0.1140 - acc: 0.9680 - val_loss: 0.2573 - val_acc: 0.9251
Epoch 4/10
4000/4000 [==============================] - 2s 494us/step - loss: 0.0930 - acc: 0.9720 - val_loss: 0.2974 - val_acc: 0.9173
Epoch 5/10
4000/4000 [==============================] - 2s 501us/step - loss: 0.0787 - acc: 0.9790 - val_loss: 0.3325 - val_acc: 0.9070
Epoch 6/10
4000/4000 [==============================] - 2s 504us/step - loss: 0.0756 - acc: 0.9785 - val_loss: 0.3346 - val_acc: 0.9087
Epoch 7/10
4000/4000 [==============================] - 2s 498us/step - loss: 0.0723 - a

In [77]:
model.predict(x_val)

array([[1.3543166e-07, 6.8838735e-06, 4.9698676e-15, ..., 5.9214018e-12,
        9.9999201e-01, 2.0079182e-13],
       [1.5148845e-07, 2.2095590e-07, 4.9754390e-13, ..., 1.5172257e-10,
        4.3803979e-08, 2.3783603e-10],
       [4.4380772e-06, 9.9999273e-01, 1.9692762e-15, ..., 1.6181231e-14,
        2.2549025e-06, 8.6437320e-14],
       ...,
       [7.6952518e-04, 9.9918586e-01, 3.2740484e-08, ..., 2.1329145e-09,
        9.6084659e-06, 1.6753145e-07],
       [2.2593276e-06, 1.8275221e-05, 6.0289583e-14, ..., 4.0921959e-09,
        5.3798012e-06, 5.8095140e-11],
       [1.6560693e-05, 8.4585281e-06, 4.6540289e-12, ..., 2.2937732e-09,
        9.9997389e-01, 1.0675727e-09]], dtype=float32)